<a href="https://colab.research.google.com/github/EliseMalvaoCarlson/Challenge2_Alura_Data_Science_TeleconX_Parte2/blob/main/TelecomX_BR_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Challenge - Telecom X - parte 2

# 🛠️ 1 - Preparação dos Dados

## 1.1 - Extração do Arquivo Tratado

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/EliseMalvaoCarlson/Challeng_TelecomX_parte2_BR/refs/heads/main/dados_tratados.csv")


In [ ]:
dados.head()

## 1.2 - Remoção de Colunas Irrelevantes

A base de dados importada já estava sem a coluna customerID

## 1.3 - Encoding

# O que é Encoding?

No processamento de dados para machine learning, encoding é o processo de transformar variáveis categóricas (como “gênero” ou “tipo de contrato”) em valores numéricos que os algoritmos conseguem entender. Modelos como Random Forest ou Regressão Logística não conseguem trabalhar diretamente com texto — eles precisam de números.

### 1.3.1 - Identificar as colunas categóricas

In [ ]:
colunas_categoricas = ['gender', 'Partner', 'Dependents', 'PhoneService',
                       'MultipleLines', 'InternetService', 'OnlineSecurity',
                       'OnlineBackup', 'DeviceProtection', 'TechSupport',
                       'StreamingTV', 'StreamingMovies', 'Contract',
                       'PaperlessBilling', 'PaymentMethod']

### 1.3.2 - Aplicar get_dummies()

**O que é get_dummies()?**

A função get_dummies() do pandas é usada para codificar variáveis categóricas em formato numérico. Isso é essencial para que algoritmos de machine learning possam processar essas informações

In [ ]:
# Codifica variáveis categóricas (drop_first=True para evitar multicolinearidade)
dados_encoded = pd.get_dummies(dados, columns=colunas_categoricas, drop_first=True)

### 1.3.3 - Verificação da Proporção de Evasão

In [ ]:
# Proporção de churn
y = dados['Churn']
proporcao_churn = y.value_counts(normalize=True).round(2) * 100

# Impressão no console
print("Proporção de clientes:")
print(f"Ativos (Churn = 0): {proporcao_churn[0]}%")
print(f"Evadidos (Churn = 1): {proporcao_churn[1]}%")

# Visualização aprimorada
plt.figure(figsize=(6, 4))
cores = ['blue', '#F44336']  # verde e vermelho mais suaves
proporcao_churn.plot(kind='bar', color=cores)

# Rótulos e título
plt.xticks(ticks=[0, 1], labels=['Ativos', 'Evadidos'], rotation=0)
plt.title('Distribuição das Classes (Churn)', fontsize=14)
plt.ylabel('% de Clientes', fontsize=12)
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Adiciona os valores nas barras
for i, valor in enumerate(proporcao_churn):
    plt.text(i, valor + 2, f'{valor:.1f}%', ha='center', fontsize=12)

plt.tight_layout()
plt.show()


### 1.3.4 - Balanceamento de Classes (opcional)

Seria necessário se a proporção de churn for muito baixa, algo como Ativos (Churn = 0): 90% / Evadidos (Churn = 1): 10%, mas como obtivemos valores mais altos, não será necessário.

### 1.3.5 - Normalização ou Padronização (se necessário)

Vamos pular essa etapa porque vamos usar Random Forest agora.

# 🎯 2 - Correlação e Seleção de Variáveis

## 2.1 - Análise de Correlação - Matriz de Correlação entre variáveis numéricas

In [ ]:
corr = dados_encoded.corr().round(4)

**O que é .corr()?**

A função .corr() calcula a matriz de correlação entre todas as variáveis numéricas do DataFrame dados_encoded. A correlação mede o grau de associação entre duas variáveis, com valores que variam de:

+1: correlação positiva perfeita

0: nenhuma correlação

–1: correlação negativa perfeita

**E o .round(4)?**

Esse método arredonda os valores da matriz de correlação para quatro casas decimais, facilitando a leitura e a visualização dos resultados.

**Resultado: corr**

O objeto corr gerado é uma tabela quadrada onde cada célula representa a correlação entre duas variáveis. Por exemplo:

corr['Churn']['tenure'] mostra a correlação entre tempo de contrato e evasão.

corr['Churn']['Contract_Two year'] mostra a correlação entre contratos bienais e churn.



In [ ]:
corr['Churn'].sort_values(ascending=False)

In [ ]:
# 1. Correlações com 'Churn'
corr_churn = corr['Churn'].drop('Churn')

# 2. Ordenar por correlação absoluta
correlacoes_ordenadas = corr_churn.reindex(
    corr_churn.abs().sort_values(ascending=False).index
)

# 3. Obter o limite da 10ª maior correlação (com empates)
top_n = 10
limite = correlacoes_ordenadas.abs().unique()[top_n - 1]
variaveis_relevantes = correlacoes_ordenadas[correlacoes_ordenadas.abs() >= limite].index.tolist()

# 4. Incluir 'Churn' na lista
variaveis_plot = ['Churn'] + variaveis_relevantes

# 5. Criar submatriz de correlação
submatriz_corr = corr.loc[variaveis_plot, variaveis_plot]

# 6. Plotar heatmap
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(
    submatriz_corr,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    vmin=-1, vmax=1,
    linewidths=0.5,
    linecolor='gray',
    cbar_kws={"shrink": 0.8}
)
plt.title('Submatriz das Maiores Correlações com Churn', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 2.2 - Análises Direcionadas

### 2.2.1 - Tempo de contrato × Evasão

In [ ]:
# Convert 'Churn' to categorical for plotting
dados['Churn'] = dados['Churn'].astype('category')

plt.figure(figsize=(8, 5))  # Tamanho do gráfico
sns.boxplot(x='Churn', y='tenure', data=dados, hue='Churn', palette={0.0: '#4CAF50', 1.0: '#F44336'}, legend=False)  # Cores personalizadas
plt.title('Tempo de Contrato x Churn', fontsize=14, fontweight='bold')
plt.xlabel('Churn (0 = Não, 1 = Sim)', fontsize=12)
plt.ylabel('Meses de Contrato', fontsize=12)
# Adjust xticks to match categorical values
plt.xticks(ticks=[0, 1], labels=['Não Evadiu', 'Evadiu'], fontsize=11)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

### 2.2.2 - Total Gasto × Evasão

In [ ]:
sns.boxplot(x='Churn', y='Charges.Total', data=dados)
plt.title('Total Gasto x Churn')
plt.xlabel('Churn (0 = Não, 1 = Sim)')
plt.ylabel('Valor Total Gasto')
plt.show()

# 🤖 3 - Modelagem Preditiva

## 3.1 - Separação de dados

In [ ]:
from sklearn.model_selection import train_test_split

X = dados_encoded.drop(columns=['Churn'])  # já está sem customerID
y = dados_encoded['Churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

## 3.2 - Criação de modelos

### 3.2.1 - Treinamento com Random Forest

**O que é Random Forest?**

Random Forest é um algoritmo de aprendizado supervisionado baseado em árvores de decisão. Ele cria várias árvores (daí o “forest”) e combina suas previsões para melhorar a precisão e reduzir o risco de overfitting.

**O que acontece após o treinamento?**

O modelo aprende padrões nos dados para prever a variável alvo (Churn). Ele será testado depois com dados novos (X_test) para verificar se consegue generalizar bem.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_rf = RandomForestClassifier(random_state=42)
modelo_rf.fit(X_train, y_train)

### 3.2.2 - Treinamento com Regressão Logística - etapa 1: Separar variáveis numéricas para normalizar apenas onde necessário

**A Regressão Logística é um modelo estatístico usado para prever variáveis**
binárias, como o churn (evasão de clientes), que pode ser 0 (permaneceu) ou 1 (evadiu). Ela estima a probabilidade de um evento ocorrer com base nas variáveis explicativas.

**Etapa 1: Normalização dos dados**

In [ ]:
from sklearn.preprocessing import StandardScaler

# Identificar colunas numéricas
numericas = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Normalizar os dados para regressão logística
scaler = StandardScaler()
X_norm = X.copy()
X_norm[numericas] = scaler.fit_transform(X_norm[numericas])

# Dividir treino/teste com os dados normalizados
Xn_train, Xn_test, yn_train, yn_test = train_test_split(
    X_norm, y, test_size=0.3, random_state=42, stratify=y # proporção de 70% para treino e 30% para teste
)

### 3.2.2 - Treinamento com Regressão Logística - etapa 2: Treinar Regressão Logística com dados normalizados

**Os dados normalizados foram divididos em treino e teste:**



In [ ]:
from sklearn.linear_model import LogisticRegression

modelo_log = LogisticRegression(max_iter=1000, random_state=42)
modelo_log.fit(Xn_train, yn_train)

### 3.2.3 - Justificativa de escolha dos modelos para análise

###  Random Forest

O modelo **Random Forest** foi adotado como uma abordagem baseada em árvores de decisão. Por sua natureza, ele é robusto a variáveis com diferentes escalas e tipos, o que elimina a necessidade de normalização dos dados. Essa característica foi considerada na etapa de pré-processamento, onde a padronização foi propositalmente omitida. Apesar de sua alta capacidade de aprendizado, o modelo apresentou sinais de **overfitting**, com desempenho quase perfeito no treino e queda significativa no teste, o que compromete sua capacidade de generalização.

---

###  Regressão Logística

A **Regressão Logística** foi escolhida como modelo linear base para prever a evasão de clientes. Por ser sensível à escala das variáveis, foi aplicada a **normalização com StandardScaler** nas colunas numéricas antes do treinamento. Essa escolha se justifica por ser um dos algoritmos mais utilizados em problemas de classificação binária, como o de churn. Além de ser simples e eficiente, ela fornece **coeficientes interpretáveis**, permitindo entender o impacto de cada variável na probabilidade de evasão.

Embora o modelo ainda não tenha sido explorado em profundidade no curso, sua configuração foi feita com base em **referências confiáveis e suporte técnico**, garantindo que os ajustes necessários — como normalização e balanceamento — fossem corretamente aplicados.

---

###  Comparação entre os modelos

A comparação entre Random Forest e Regressão Logística permitiu avaliar o desempenho entre técnicas **não-lineares** e **lineares** no contexto da previsão de churn. A Regressão Logística demonstrou maior consistência entre treino e teste, melhor recall e F1-score, além de ser mais confiável para aplicação prática. Já o Random Forest, apesar de sua alta acurácia no treino, mostrou dificuldade de generalização, reforçando a importância de ajustes como o GridSearchCV.



## 3.3 - Avaliação dos Modelos

### 3.3.1 - Métricas: acurácia, precisão, recall e F1-score.

O que são essas métricas?

Essas quatro métricas são usadas para **avaliar o desempenho de modelos de classificação**, especialmente em problemas binários como o churn (evasão de clientes):

| Métrica       | Definição                                                                 | Interpretação prática no notebook |
|---------------|---------------------------------------------------------------------------|-----------------------------------|
| **Acurácia**  | Proporção de previsões corretas sobre o total de casos                    | Mede o desempenho geral do modelo |
| **Precisão**  | Proporção de casos previstos como positivos que realmente são positivos   | Indica se o modelo evita falsos positivos |
| **Recall**    | Proporção de casos positivos que foram corretamente identificados         | Mede a capacidade de detectar clientes que realmente evadiram |
| **F1-score**  | Média harmônica entre precisão e recall                                   | Equilíbrio entre acertos e cobertura |

---

### Como foram aplicadas no notebook?

O notebook calculou essas métricas para os dois modelos — **Random Forest** e **Regressão Logística** — tanto no conjunto de **treino** quanto no de **teste**:

```python
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
```

As previsões foram feitas com `.predict()` e os resultados foram organizados em um `DataFrame` chamado `df_metricas`, arredondado para três casas decimais.

---

###  Resultados observados

####  Random Forest
- **Treino**: Acurácia de 0.998, Recall de 0.995 → quase perfeito (indício de overfitting)
- **Teste**: Acurácia de 0.784, Recall de 0.494 → caiu bastante, errou mais da metade dos churns

####  Regressão Logística
- **Treino**: Acurácia de 0.811, Recall de 0.563 → desempenho mais realista
- **Teste**: Acurácia de 0.798, Recall de 0.545 → melhor equilíbrio e generalização

---

###  Conclusão

Essas métricas permitiram comparar os modelos de forma objetiva. O **Random Forest** teve alta acurácia no treino, mas baixo recall no teste, indicando overfitting. Já a **Regressão Logística** mostrou desempenho mais consistente e maior capacidade de identificar clientes que realmente evadiram — o que é essencial para o objetivo do projeto.


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score
)
import pandas as pd

# Previsões dos dois modelos - TESTE
y_pred_rf = modelo_rf.predict(X_test)
y_pred_log = modelo_log.predict(Xn_test)

# Previsões dos dois modelos - TREINO
y_pred_rf_train = modelo_rf.predict(X_train)
y_pred_log_train = modelo_log.predict(Xn_train)

# Dicionários com as métricas principais - TESTE
metricas_rf_test = {
    'Modelo': 'Random Forest - Teste',
    'Acurácia': accuracy_score(y_test, y_pred_rf),
    'Precisão': precision_score(y_test, y_pred_rf),
    'Recall': recall_score(y_test, y_pred_rf),
    'F1-Score': f1_score(y_test, y_pred_rf)
}

metricas_log_test = {
    'Modelo': 'Regressão Logística - Teste',
    'Acurácia': accuracy_score(yn_test, y_pred_log),
    'Precisão': precision_score(yn_test, y_pred_log),
    'Recall': recall_score(yn_test, y_pred_log),
    'F1-Score': f1_score(yn_test, y_pred_log)
}


# Dicionários com as métricas principais - TREINO
metricas_rf_train = {
    'Modelo': 'Random Forest - Treino',
    'Acurácia': accuracy_score(y_train, y_pred_rf_train),
    'Precisão': precision_score(y_train, y_pred_rf_train),
    'Recall': recall_score(y_train, y_pred_rf_train),
    'F1-Score': f1_score(y_train, y_pred_rf_train)
}

metricas_log_train = {
    'Modelo': 'Regressão Logística - Treino',
    'Acurácia': accuracy_score(yn_train, y_pred_log_train),
    'Precisão': precision_score(yn_train, y_pred_log_train),
    'Recall': recall_score(yn_train, y_pred_log_train),
    'F1-Score': f1_score(yn_train, y_pred_log_train)
}

# Criar DataFrame com os dois modelos
df_metricas = pd.DataFrame([metricas_rf_test, metricas_log_test, metricas_rf_train, metricas_log_train])
df_metricas.set_index('Modelo', inplace=True)
df_metricas = df_metricas.round(3)

# Mostrar as métricas em tabela
print("\n📊 Comparativo das Métricas Principais:")
df_metricas


### 3.3.1 - Métrica: matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

# --- Previsões para o conjunto de TESTE ---
y_pred_rf_test = modelo_rf.predict(X_test)
y_pred_log_test = modelo_log.predict(Xn_test)

# --- Previsões para o conjunto de TREINO ---
y_pred_rf_train = modelo_rf.predict(X_train)
y_pred_log_train = modelo_log.predict(Xn_train)

print("--- MATRIZES DE CONFUSÃO ---")

# --- Matriz Random Forest - TESTE ---
print("\n📊 Matriz de Confusão - Random Forest (Teste)")
print(pd.DataFrame(confusion_matrix(y_test, y_pred_rf_test),
                   index=['Atual: Permaneceu', 'Atual: Evadiu'],
                   columns=['Previsto: Permaneceu', 'Previsto: Evadiu']))

# --- Matriz Random Forest - TREINO ---
print("\n📊 Matriz de Confusão - Random Forest (Treino)")
print(pd.DataFrame(confusion_matrix(y_train, y_pred_rf_train),
                   index=['Atual: Permaneceu', 'Atual: Evadiu'],
                   columns=['Previsto: Permaneceu', 'Previsto: Evadiu']))

print("\n" + "=" * 60 + "\n")  # Divisor maior entre modelos

# --- Matriz Regressão Logística - TESTE ---
print("📊 Matriz de Confusão - Regressão Logística (Teste)")
print(pd.DataFrame(confusion_matrix(yn_test, y_pred_log_test),
                   index=['Atual: Permaneceu', 'Atual: Evadiu'],
                   columns=['Previsto: Permaneceu', 'Previsto: Evadiu']))

# --- Matriz Regressão Logística - TREINO ---
print("\n📊 Matriz de Confusão - Regressão Logística (Treino)")
print(pd.DataFrame(confusion_matrix(yn_train, y_pred_log_train),
                   index=['Atual: Permaneceu', 'Atual: Evadiu'],
                   columns=['Previsto: Permaneceu', 'Previsto: Evadiu']))


### 3.3.2 - Análise crítica

## ✅ **Resumo geral das métricas**

| Modelo                      | Acurácia | Precisão | Recall | F1-Score |
| --------------------------- | -------- | -------- | ------ | -------- |
| **Random Forest - Treino**  | 0.998    | 0.996    | 0.995  | 0.996    |
| **Random Forest - Teste**   | 0.784    | 0.617    | 0.494  | 0.549    |
| **Reg. Logística - Treino** | 0.811    | 0.671    | 0.563  | 0.613    |
| **Reg. Logística - Teste**  | 0.798    | 0.640    | 0.545  | 0.589    |

---

## 🔍 **Análise detalhada por modelo**

### 🌲 **Random Forest**

#### 🟩 Ponto positivo:

* Acertou 1380 dos 1552 clientes que permaneceram.
* Boa precisão no teste (0.617), ou seja, quando prevê churn, costuma acertar.

#### 🟥 Ponto preocupante:

* **Desempenho em treino altíssimo (quase 100%)** e queda forte no teste.
* Recall no teste é baixo (0.494), ou seja, **erra mais de 50% dos clientes que evadiram**.
* Matriz de confusão do treino mostra apenas **11 erros em mais de 5000 registros** — isso é um **sinal claro de overfitting**.

#### 📉 Conclusão:

> **Random Forest memorizou os dados de treino** e teve dificuldade de generalizar no conjunto de teste. Apesar de sua alta performance no treino, seu desempenho real é limitado.

---

### 📈 **Regressão Logística**

#### 🟩 Ponto forte:

* Manteve **boa consistência entre treino e teste** — ou seja, generaliza melhor.
* No teste:

  * **Maior recall (0.545)** → identificou mais clientes que realmente evadiram
  * **Melhor F1-score (0.589)** → equilíbrio mais robusto entre precisão e recall

#### 🟨 Considerações:

* No treino, teve desempenho inferior ao Random Forest, mas **sem exagero**.
* Apresentou 737 acertos de churn e 571 falsos negativos (matriz de confusão treino), o que é **realista para modelos simples e interpretáveis**.

#### 📉 Conclusão:

> Embora menos potente que Random Forest, a Regressão Logística mostra **desempenho estável, interpretável e mais confiável para aplicação prática.**

---

## 🧠 **Análise comparativa (crítica)**

| Critério                  | Random Forest        | Regressão Logística | Melhor modelo |
| ------------------------- | -------------------- | ------------------- | ------------- |
| Acurácia no teste         | 0.784                | **0.798**           | 🔹 Logística  |
| Recall (teste - churn)    | 0.494                | **0.545**           | 🔹 Logística  |
| F1-Score (teste)          | 0.549                | **0.589**           | 🔹 Logística  |
| Consistência treino/teste | ❌ (muito gap)        | ✅ (coerente)        | 🔹 Logística  |
| Overfitting detectado?    | ✅ Sim                | ❌ Não               | 🔹 Logística  |
| Facilidade de explicação  | Média (importâncias) | Alta (coeficientes) | 🔹 Logística  |


### 3.3.3 - Justificativa da escolha do modelo

Com base nas métricas de desempenho obtidas até esta etapa, o modelo de **Regressão Logística** apresentou resultados superiores ao Random Forest em todos os principais indicadores: acurácia, precisão, recall e F1-score. Destaca-se, sobretudo, o **maior recall**, essencial para a identificação de clientes com risco real de evasão — objetivo central deste projeto.

O modelo **Random Forest**, por sua vez, apesar de apresentar boa performance em teste, demonstrou sinais evidentes de **overfitting**, com desempenho quase perfeito no conjunto de treino. Esse comportamento sugere baixa capacidade de generalização para novos dados.

**Assim, até o momento, a Regressçao Logística é a opção mais adequada dentre os modelos treinados inicialmente.**

Entretanto, essa análise crítica abre espaço para possíveis melhorias. Em uma próxima etapa, será conduzido um **aprofundamento técnico com refinamento dos modelos, visando corrigir distorções como overfitting e melhorar a sensibilidade na detecção de churn.**

Esse processo permitirá validar se a escolha atual permanece como a mais indicada ou se outro modelo, após ajustes, poderá apresentar desempenho mais robusto.

### 3.3.4 - Overfitting ou Underfitting

### 🔍 Random Forest

**Conclusão:**
> Forte indício de **Overfitting**.  
O modelo Random Forest performou quase perfeitamente nos dados de treino, mas teve **queda significativa** no teste. Isso indica que o modelo possivelmente **"memorizou" os dados de treino**, comprometendo a **generalização**.

---

### 🔍 Regressão Logística

**Conclusão:**
> A diferença entre treino e teste é **muito menor**, o que indica **boa generalização**.  
A Regressão Logística **não sofre de overfitting severo**.  
Entretanto, o **recall moderado** sugere que o modelo ainda **perde quase metade dos clientes que realmente evadem**. Isso pode indicar um possível **underfitting da classe minoritária**, sem ser um underfitting generalizado.

---

## 📌 Resumo Geral

- **Random Forest**: Apresentou **overfitting** — o modelo está excessivamente ajustado aos dados de treino.
- **Regressão Logística**: Apresentou **consistência** entre treino e teste. Ainda que o recall seja moderado, **não há overfitting severo**. Pode haver uma **dificuldade em capturar a classe minoritária** (churn).

# 📋 4 - Interpretação e Conclusões

## 4.1 - Random Forest – Importância das Variáveis

Objetivo: entender quais variáveis mais impactam a decisão do modelo.

In [ ]:
importances = modelo_rf.feature_importances_
variaveis = X.columns

importancia_df = pd.DataFrame({
    'Variável': variaveis,
    'Importância': importances
}).sort_values(by='Importância', ascending=False)

# Visualizar as 10 variáveis mais importantes
print(importancia_df.head(10))

# Gráfico
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(9, 6))
sns.barplot(x='Importância', y='Variável', data=importancia_df.head(10))
plt.title('Top 10 Variáveis mais Importantes - Random Forest')
plt.tight_layout()
plt.show()

## 4.2 - Regressão Logística – Coeficientes das Variáveis

Objetivo: verificar a influência (positiva ou negativa) de cada variável na evasão.

Interpretação dos coeficientes:

- Coeficiente positivo: aumenta a chance de churn;

- Coeficiente negativo: reduz a chance de churn.

In [ ]:
coeficientes = modelo_log.coef_[0]
variaveis_log = Xn_train.columns

coef_df = pd.DataFrame({
    'Variável': variaveis_log,
    'Coeficiente': coeficientes
}).sort_values(by='Coeficiente', key=abs, ascending=False)

# Exibir as variáveis com maior peso absoluto
print(coef_df.head(10))

# Gráfico
plt.figure(figsize=(10, 6))
sns.barplot(x='Coeficiente', y='Variável', data=coef_df.head(10))
plt.title('Top 10 Coeficientes - Regressão Logística')
plt.tight_layout()
plt.show()

## 4.3 - Análise crítica dos modelos com refinamentos

### 4.3.1 - Revisão Crítica da Modelagem

Com base na Etapa 3, foi possível identificar que o modelo de Random Forest apresentava sinais de overfitting e que a Regressão Logística, embora mais estável, ainda poderia melhorar o recall — métrica crítica em problemas de churn.

Dessa forma, nesta etapa foram aplicadas técnicas de refinamento e ajustes para validar se melhorias nos modelos poderiam gerar ganhos expressivos, sem perder a capacidade de generalização.

As técnicas aplicadas foram:

- Ajuste de hiperparâmetros com `GridSearchCV` no Random Forest
- Regressão Logística com `class_weight='balanced'`
- Ajuste do threshold (limiar de classificação) na Regressão Logística

### 4.3.2 - Aplicação das Melhorias Propostas

#### 4.3.2.1 - Ajuste de Hiperparâmetros no Random Forest

Para mitigar o overfitting observado no Random Forest, foi aplicado um ajuste de hiperparâmetros via GridSearchCV, otimizando a métrica de F1-score.

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Definir o grid de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Criar e treinar o GridSearchCV
grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='f1',
    verbose=1
)

# Treinar com os dados
grid_rf.fit(X_train, y_train)

# Obter o melhor modelo
melhor_rf = grid_rf.best_estimator_

# Prever no conjunto de teste
y_pred_melhor_rf = melhor_rf.predict(X_test)

# Exibir relatório e matriz de confusão
print("=== Random Forest com Hiperparâmetros Otimizados ===")
print(classification_report(y_test, y_pred_melhor_rf))
print("Matriz de Confusão:")
print(confusion_matrix(y_test, y_pred_melhor_rf))

Fitting 5 folds for each of 48 candidates, totalling 240 fits
=== Random Forest com Hiperparâmetros Otimizados ===
              precision    recall  f1-score   support

         0.0       0.84      0.89      0.86      1552
         1.0       0.64      0.51      0.57       561

    accuracy                           0.79      2113
   macro avg       0.74      0.70      0.72      2113
weighted avg       0.78      0.79      0.79      2113

Matriz de Confusão:
[[1387  165]
 [ 273  288]]


#### 4.3.2.2 - Regr. Logística com class_weight='balanced'

Considerando o custo de falsos negativos em churn, foi testado o uso de class_weight='balanced' na Regressão Logística. A estratégia prioriza o aprendizado da classe minoritária, mesmo com proporção moderada (26.5%).

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Treinamento com balanceamento de classes
modelo_log_bal = LogisticRegression(max_iter=1000, class_weight='balanced')
modelo_log_bal.fit(Xn_train, yn_train)

# Previsão
y_pred_log_bal = modelo_log_bal.predict(Xn_test)

# Avaliação
print("=== Regressão Logística com class_weight='balanced' ===")
print(classification_report(yn_test, y_pred_log_bal))
print("Matriz de Confusão:")
print(confusion_matrix(yn_test, y_pred_log_bal))

=== Regressão Logística com class_weight='balanced' ===
              precision    recall  f1-score   support

         0.0       0.91      0.71      0.80      1552
         1.0       0.50      0.80      0.61       561

    accuracy                           0.73      2113
   macro avg       0.70      0.75      0.71      2113
weighted avg       0.80      0.73      0.75      2113

Matriz de Confusão:
[[1103  449]
 [ 113  448]]


#### 4.3.2.3 - Ajuste de Threshold com Base na Curva de Precisão-Recall

Por fim, foi ajustado o threshold de decisão da Regressão Logística tradicional. O objetivo foi encontrar o ponto com melhor F1-score mantendo recall ≥ 0.65. Essa abordagem visa melhorar a sensibilidade sem comprometer excessivamente a precisão.

In [53]:
# Gerar probabilidades com o modelo de Regressão Logística
y_probs = modelo_log.predict_proba(Xn_test)[:, 1]  # Probabilidade da classe 1 (churn)

# Obter precisão, recall e thresholds da curva
prec, rec, thresholds = precision_recall_curve(yn_test, y_probs)

# Calcular F1-score para cada ponto da curva
f1_scores = 2 * (prec * rec) / (prec + rec + 1e-6)  # evitar divisão por zero

# Filtrar apenas os thresholds com recall acima de 0.65
validos = rec >= 0.65

# Selecionar o melhor threshold com base no maior F1-score entre os válidos
if np.any(validos):
    idx_max_f1 = np.argmax(f1_scores * validos)
    threshold_otimizado = thresholds[idx_max_f1]
else:
    threshold_otimizado = 0.5  # fallback padrão

# Aplicar novo threshold às probabilidades
y_pred_threshold = (y_probs >= threshold_otimizado).astype(int)

# Avaliação
print(f"=== Regressão Logística com Novo Threshold Otimizado (threshold = {threshold_otimizado:.2f}) ===")
print(classification_report(yn_test, y_pred_threshold))
print("Matriz de Confusão:")
print(confusion_matrix(yn_test, y_pred_threshold))


=== Regressão Logística com Novo Threshold Otimizado (threshold = 0.31) ===
              precision    recall  f1-score   support

         0.0       0.90      0.76      0.83      1552
         1.0       0.54      0.76      0.63       561

    accuracy                           0.76      2113
   macro avg       0.72      0.76      0.73      2113
weighted avg       0.80      0.76      0.78      2113

Matriz de Confusão:
[[1186  366]
 [ 132  429]]


## 4.3 - Comparativo Final dos Modelos

A seguir, são apresentados os resultados comparativos obtidos com esses aprimoramentos.

| Modelo                                 | Acurácia | Precisão | Recall    | F1-Score  | FN (evadiu, mas não previsto) |
| -------------------------------------- | -------- | -------- | --------- | --------- | ----------------------------- |
| **Random Forest (original)**           | 0.784    | 0.617    | 0.494     | 0.549     | 284                           |
| **Random Forest (ajustado)**           | 0.790    | 0.640    | 0.510     | 0.570     | 273                           |
| **Regressão Logística (original)**     | 0.798    | 0.640    | 0.545     | 0.589     | 255                           |
| **Regr. Logística com `balanced`**     | 0.730    | 0.500    | **0.800** | 0.610     | **113**                       |
| **Regr. Logística com threshold 0.31** | 0.760    | 0.540    | 0.765     | **0.630** | 132                           |

## 4.4 - Justificativa do Modelo Final Escolhido

Após aplicar técnicas de refinamento, como ajuste de threshold e balanceamento de classes, o modelo que demonstrou o melhor desempenho geral foi a Regressão Logística com threshold ajustado para 0.31.

Esse modelo alcançou um F1-score de 0.63, recall de 0.765 e conseguiu reduzir o número de clientes evasores não identificados para 132 — um dos melhores resultados entre todos os testes realizados.

A escolha está alinhada com o objetivo principal do projeto: identificar clientes com risco de churn de forma antecipada, sem sacrificar completamente a precisão ou a acurácia global.

## 4.5 - Conclusões

📌 Principais fatores associados ao churn
Com base nas análises dos modelos Random Forest e Regressão Logística, os fatores mais relevantes foram:

🔍 Random Forest – Variáveis com maior importância:
Charges.Total (gastos totais)

Tenure (tempo como cliente)

Charges.Monthly e Charges.Daily

Tipo de contrato (Contract_Two year, Contract_One year)

Forma de pagamento (PaymentMethod_Electronic check)

Tipo de internet (InternetService_Fiber optic)

Fatura digital (PaperlessBilling_Yes)

Gênero masculino (gender_Male)

Essas variáveis contribuíram para reduzir a impureza nas árvores, influenciando fortemente a classificação.

📈 Regressão Logística – Interpretação dos coeficientes:
Tenure e Contract_Two year: coeficientes negativos → reduzem a chance de churn

InternetService_Fiber optic, Charges.Total, PaperlessBilling_Yes, PaymentMethod_Electronic check: coeficientes positivos → aumentam a chance de churn

TechSupport_Yes e PhoneService_Yes: coeficientes negativos → indicam retenção

🧠 Conclusão Técnica
Mesmo em sua versão inicial, a Regressão Logística já superava o Random Forest em recall e F1-score, mostrando maior capacidade de generalização. Após os refinamentos, ela se consolidou como o modelo mais eficaz para detectar churn com sensibilidade e estabilidade.

Ambos os modelos destacaram variáveis como tenure, tipo de contrato e gastos acumulados, mas a Regressão Logística ofereceu interpretações mais claras, facilitando decisões orientadas por dados.

💡 Estratégias de Retenção sugeridas
Incentivar contratos de longo prazo com benefícios progressivos

Monitorar clientes com altos gastos e oferecer suporte proativo

Avaliar perfis com fatura digital e débito eletrônico (maior risco)

Reforçar suporte técnico, especialmente para usuários de fibra ótica

Criar campanhas de fidelização para clientes com menos de 6 meses de casa

✅ Encerramento
O projeto foi concluído com sucesso, integrando técnicas de pré-processamento, modelagem supervisionada e análise interpretativa. Os resultados obtidos oferecem à empresa uma base sólida para reduzir churn de forma estratégica e orientada por dados.